In [ ]:
!pip install tqdm torchvision librosa wandb matplotlib

In [ ]:
# Train syncnet from scrach
!python transformer_syncnet_train.py --data_root lrs2_preprocessed_28/ \
--checkpoint_dir checkpoints/syncnet_checkpoint

In [5]:
# Train original wav2lip from scrach
!python transformer_wav2lip_train.py --data_root lrs2_preprocessed_28/ \
--checkpoint_dir checkpoints \
--syncnet_checkpoint_path checkpoints/syncnet_checkpoint/checkpoint_step000018000.pth

use_cuda: False
/opt/anaconda3/envs/for_wav2lip/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
total trainable params 71644099
Load checkpoint from: checkpoints/syncnet_checkpoint/checkpoint_step000018000.pth
/Users/eddyma/DEV/Github/Wav2Lip/transformer_wav2lip_train.py:539: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weight

## Helper functions

In [6]:
# Generate the training file
import os

def get_subfolders(directory):
    return [os.path.join(directory, name) for name in os.listdir(directory)
            if os.path.isdir(os.path.join(directory, name))]

for subdir in get_subfolders('lrs2_preprocessed_28'):
  for subdir2 in get_subfolders(subdir):
    for root, dirs, files in os.walk(subdir2):
        print(subdir2)
        # Extract the desired portion (last two parts of the path)
        desired_portion = os.path.join(*subdir2.split(os.sep)[-2:])

        # # Path to the output text file
        output_file_path = "output.txt"

        # # Append the extracted portion to the text file as a new line
        with open(output_file_path, 'a') as f:
             f.write(desired_portion + '\n')

        print(f"Appended '{desired_portion}' to {output_file_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/lrs2_preprocessed_28'